# Initial data Exploration
Data is available in `CSV` file.

In [ ]:
# imports
import pandas as pd

In [ ]:
# install python package from the src/ folder (then you need to restart the kernel!)
!cd ../.. & python setup.py build & python setup.py install

In [ ]:
import src

In [ ]:
FILENAME = "../../../data/raw/continental/continental_preliminary.csv"

In [ ]:
# import data
df = pd.read_csv(FILENAME, header=0,
    names=[
        "Id",
        "Timestamp",
        "SerialNumber",
        "Station",
        "StationType",
        "StationNumber",
        "Material",
        "TextDescription",
        "TestValue",
        "TestResult",
        "USL",
        "LSL",
        "Format"
    ]
)

In [ ]:
df.describe()